Да бисте покренули следеће бележнице, ако то још нисте урадили, потребно је да унутар .env фајла подесите openai кључ као `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Следеће, учитаћемо Индекс уграђивања у Pandas Dataframe. Индекс уграђивања је сачуван у JSON фајлу под називом `embedding_index_3m.json`. Индекс уграђивања садржи уграђивања за сваки од YouTube транскрипата до краја октобра 2023. године.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Затим ћемо креирати функцију под називом `get_videos` која ће претраживати Индекс уграђивања за упит. Функција ће вратити 5 најбољих видео записа који су најсличнији упиту. Функција ради на следећи начин:

1. Прво се креира копија Индекса уграђивања.
2. Затим се израчунава уграђивање за упит користећи OpenAI Embedding API.
3. Потом се у Индексу уграђивања креира нови стубац под називом `similarity`. Стубац `similarity` садржи косинусну сличност између уграђивања упита и уграђивања за сваки видео сегмент.
4. Након тога, Индекс уграђивања се филтрира по стубцу `similarity`. Индекс уграђивања се филтрира тако да укључује само видео записе који имају косинусну сличност већу или једнаку 0.75.
5. На крају, Индекс уграђивања се сортира по стубцу `similarity` и враћа се 5 најбољих видео записа.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ова функција је веома једноставна, она само исписује резултате претраге.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Прво, Индекс уграђивања се учитава у Pandas Dataframe.
2. Затим се корисник подстиче да унесе упит.
3. Након тога се позива функција `get_videos` да претражи Индекс уграђивања за упит.
4. На крају, позива се функција `display_results` да прикаже резултате кориснику.
5. Корисник затим уноси нови упит. Овај процес се наставља док корисник не унесе `exit`.

![](../../../../translated_images/sr/notebook-search.1e320b9c7fcbb0bc.png)

Бићете подстакнути да унесете упит. Унесите упит и притисните ентер. Апликација ће вратити листу видео снимака који су релевантни за упит. Апликација ће такође вратити линк до места у видео снимку где се налази одговор на питање.

Ево неколико упита које можете испробати:

- Шта је Azure Machine Learning?
- Како функционишу конволуционе неуронске мреже?
- Шта је неуронска мрежа?
- Могу ли користити Jupyter Notebooks са Azure Machine Learning?
- Шта је ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Одрицање од одговорности**:
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитетним извором. За критичне информације препоручује се професионални људски превод. Нисмо одговорни за било каква неспоразума или погрешна тумачења која произилазе из коришћења овог превода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
